In [1]:
# python imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import math
import wandb
import time

In [2]:
#torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [3]:
df = pd.read_csv("dataset_done.csv",index_col=0)
df = df.drop(columns={"index"})

In [4]:
df['ALCOHOL'] = df['ALCOHOL'].apply(lambda x : x/100)
df['SUGAR'] = df['SUGAR'].apply(lambda x : x/1000)
df['ACIDS'] = df['ACIDS'].apply(lambda x : x/100)

In [5]:
df.head()

,ALCOHOL,SUGAR,ACIDS,acid,bodied,dry,sweet,tannic,floral,citrus fruit,...,dairy_food,fatty_food,lamb_food,nutty_food,box,desert,red,rose,sparkling,white
0,0.15,0.003,0.052,0,3,0,0,2,0,0,...,0,1,1,0,1,0,0,0,0,0
1,0.15,0.003,0.052,0,3,0,0,2,0,0,...,1,0,0,1,1,0,0,0,0,0
2,0.15,0.003,0.052,0,3,0,0,2,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0.15,0.003,0.052,0,3,0,0,2,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0.12,0.016,0.049,2,0,0,2,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [79]:
class CycleGenerator(nn.Module):
    """
    Generator for CycleGAN
    """
    def __init__(self, input_channels, output_channels,hidden):
        super().__init__()
        filters = 16

        #net
        self.input = nn.Linear(input_channels, hidden)
        self.h1 = nn.Linear(hidden, hidden)
        self.h2 = nn.Linear(hidden, hidden)
        self.h3 = nn.Linear(hidden, hidden)
        self.output = nn.Linear(hidden, output_channels) 
        # modules = [nn.Linear(input_channels, hidden),
        #            nn.ReLU(),
        #            nn.Linear(hidden, hidden),
        #            nn.ReLU(),
        #            nn.Linear(hidden, hidden),
        #            nn.ReLU(),
        #            nn.Linear(hidden, hidden),
        #            nn.ReLU(),
        #            nn.Linear(hidden, output_channels),
        #            nn.ReLU()]

        # self.net = nn.Sequential(*modules)

    def forward(self, x):
        x = self.input(x)
        x = F.relu(self.h1(x))
        residual = x
        x = F.relu(self.h2(x))
        x = F.relu(self.h3(x)) + residual
        x = F.relu(self.output(x))
        return x
        # return self.net(x)


In [7]:
class CycleDiscriminator(nn.Module):
    """
    Discriminator for CycleGAN
    """

    def __init__(self, input_channels,hidden):
        super().__init__()

        self.net = nn.Sequential(
           nn.Linear(input_channels, hidden),
                   nn.ReLU(),
                   nn.Linear(hidden, hidden),
                   nn.ReLU(),
                   nn.Linear(hidden, hidden),
                   nn.ReLU(),
                   nn.Linear(hidden, 1),
                   nn.Sigmoid())

    def forward(self, x):
        return self.net(x)


In [8]:
food_columns = [x for x in df.columns if "food" in x   ]
wine_columns = [x for x in df.columns if "food" not in x   ]

In [9]:
import random
for index, row in df.iterrows():
    wine_rand = round(random.uniform(0.9, 1.2), 2)
    food_rand =  round(random.uniform(0.9, 1.2), 2)
    df.at[index,wine_columns] = row[wine_columns] * wine_rand
    df.at[index,food_columns] = row[food_columns] * food_rand

In [10]:
df

,ALCOHOL,SUGAR,ACIDS,acid,bodied,dry,sweet,tannic,floral,citrus fruit,...,dairy_food,fatty_food,lamb_food,nutty_food,box,desert,red,rose,sparkling,white
0,0.16500,0.00330,0.05720,0.00,3.30,0.0,0.00,2.20,0.0,0.00,...,0.00,1.10,1.1,0.00,1.10,0.0,0.0,0.0,0.0,0.00
1,0.15000,0.00300,0.05200,0.00,3.00,0.0,0.00,2.00,0.0,0.00,...,1.06,0.00,0.0,1.06,1.00,0.0,0.0,0.0,0.0,0.00
2,0.15900,0.00318,0.05512,0.00,3.18,0.0,0.00,2.12,0.0,0.00,...,0.00,0.00,0.0,0.00,1.06,0.0,0.0,0.0,0.0,0.00
3,0.15450,0.00309,0.05356,0.00,3.09,0.0,0.00,2.06,0.0,0.00,...,0.00,0.00,0.0,0.00,1.03,0.0,0.0,0.0,0.0,0.00
4,0.13560,0.01808,0.05537,2.26,0.00,0.0,2.26,0.00,0.0,0.00,...,0.00,0.00,0.0,0.00,1.13,0.0,0.0,0.0,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26046,0.14280,0.00595,0.06902,3.57,0.00,0.0,0.00,0.00,0.0,1.19,...,0.00,0.00,0.0,0.00,0.00,0.0,0.0,0.0,0.0,1.19
26047,0.13375,0.02675,0.05671,2.14,0.00,0.0,2.14,0.00,0.0,0.00,...,0.00,0.97,0.0,0.00,0.00,0.0,0.0,0.0,0.0,1.07
26048,0.12250,0.02450,0.05194,1.96,0.00,0.0,1.96,0.00,0.0,0.00,...,0.00,1.17,0.0,1.17,0.00,0.0,0.0,0.0,0.0,0.98
26049,0.14125,0.02825,0.05989,2.26,0.00,0.0,2.26,0.00,0.0,0.00,...,1.04,0.00,0.0,1.04,0.00,0.0,0.0,0.0,0.0,1.13


In [11]:
sweep_config = {
  "name": "NN-zadanie3",
  "method": "random",
  "metric" : {
    'name': 'loss',
    'goal': 'minimize'   
    },
 "parameters" : {
    'optimizer': {
        'values': ['sgd']
        },

    'criterion': {
        'values': ['mse']
        },

    'generator': {
        'values': [20, 30, 40, 50]
        },

    'discriminator': {
        'values': [20, 30, 40, 50]
        },

    'learning_rate_disc': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
        },
    'learning_rate_gen': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
        },

    'epochs': {
        'values': [50,100,200,500]
        },
    'cycle_lambda' :{
        'values' : [10,20,30,40,50]
    }
    }   

}

In [12]:
sweep_id = wandb.sweep(sweep_config)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Create sweep with ID: j3s1s1fz
Sweep URL: https://wandb.ai/ramang/uncategorized/sweeps/j3s1s1fz


In [13]:
# def train_cycle_gan():

#     food_data = df[food_columns]
#     wine_data = df[wine_columns]

#     n_epochs = 100
#     food_data = torch.from_numpy(food_data.to_numpy()).float().cuda()
#     wine_data = torch.from_numpy(wine_data.to_numpy()).float().cuda()
    
#     dataset_params = {'batch_size': 1, 'shuffle': False}
#     food_loader = torch.utils.data.DataLoader(food_data, **dataset_params)
#     wine_loader = torch.utils.data.DataLoader(wine_data, **dataset_params)
    
#     generator_x2y = CycleGenerator(input_channels=len(food_columns), output_channels=len(wine_columns),hidden=config.generator).cuda()
#     generator_y2x = CycleGenerator(input_channels=len(wine_columns), output_channels=len(food_columns),hidden=config.generator).cuda()
#     discriminator_x = CycleDiscriminator(input_channels=len(food_columns),hidden=config.discriminator).cuda()
#     discriminator_y = CycleDiscriminator(input_channels=len(wine_columns),hidden=config.discriminator).cuda()

#     # generator_optimizer = torch.optim.Adam(list(generator_x2y.parameters()) + list(generator_y2x.parameters()), lr=2e-4,
#     #                                        betas=(0.5, 0.999))
#     generator_optimizer = optim.SGD(list(generator_x2y.parameters()) + list(generator_y2x.parameters()), lr=config.learning_rate_gen, momentum=0.9)                                    
#     #discriminator_x_optimizer = torch.optim.Adam(discriminator_x.parameters(), lr=2e-4, betas=(0.5, 0.999))
    
#     discriminator_x_optimizer = optim.SGD(discriminator_x.parameters(), lr=config.learning_rate_disc, momentum=0.9)   
#     discriminator_y_optimizer = optim.SGD(discriminator_y.parameters(), lr=config.learning_rate_disc, momentum=0.9)   
#     #discriminator_y_optimizer = torch.optim.Adam(discriminator_y.parameters(), lr=2e-4, betas=(0.5, 0.999))

#     # Schedulers
#     num_batches = len(food_loader)
#     max_steps = max(1, n_epochs * num_batches)

#     def scheduler(epoch):
#         return max(0, (max_steps - epoch * num_batches) / max_steps)

#     generator_scheduler = optim.lr_scheduler.LambdaLR(generator_optimizer, scheduler)
#     discriminator_x_scheduler = optim.lr_scheduler.LambdaLR(discriminator_x_optimizer, scheduler)
#     discriminator_y_scheduler = optim.lr_scheduler.LambdaLR(discriminator_y_optimizer, scheduler)
#     mse_loss = nn.MSELoss()
#     l1_loss = nn.L1Loss()
#     zeros, ones = torch.tensor(0.).cuda(), torch.tensor(1.).cuda()
#     cycle_lambda = config.cycle_lambda 
   


#     for epoch in range(n_epochs):
#         for x_batch, y_batch in zip(food_loader, wine_loader):
#             # Training generators x2y and y2x
#             x_fake = generator_y2x(y_batch)
#             d_x_fake = discriminator_x(x_fake)
#             generator_x_loss = mse_loss(d_x_fake, ones)
            
#             y_fake = generator_x2y(x_batch)
#             d_y_fake = discriminator_y(y_fake)
#             generator_y_loss = mse_loss(d_y_fake, ones)
            
#             x_recovered = generator_y2x(y_fake)
#             y_recovered = generator_x2y(x_fake)
#             cycle_loss = l1_loss(x_recovered, x_batch) + l1_loss(y_recovered, y_batch)
            
#             generator_loss = generator_x_loss + generator_y_loss + cycle_lambda * cycle_loss
#             generator_optimizer.zero_grad()
#             generator_loss.backward()
#             generator_optimizer.step()                
            
#             # Training discriminator for x
#             x_fake = generator_y2x(y_batch)
#             d_x_fake = discriminator_x(x_fake)
#             d_x_true = discriminator_x(x_batch)
#             discriminator_x_loss = mse_loss(d_x_true, ones ) + mse_loss(d_x_fake, zeros)
            
#             discriminator_x_optimizer.zero_grad()
#             discriminator_x_loss.backward()
#             discriminator_x_optimizer.step()

#             # Training discriminator for y
#             y_fake = generator_x2y(x_batch)
#             d_y_fake = discriminator_y(y_fake)
#             d_y_true = discriminator_y(y_batch)
#             discriminator_y_loss = mse_loss(d_y_true, ones) + mse_loss(d_y_fake, zeros)

#             discriminator_y_optimizer.zero_grad()
#             discriminator_y_loss.backward()
#             discriminator_y_optimizer.step()
#         generator_scheduler.step()
#         discriminator_x_scheduler.step()
#         discriminator_y_scheduler.step()

#         print(f"[{100*(epoch+1)/n_epochs:.2f}%] Epoch {epoch + 1} - Time elapsed: {time.time() - epoch_start:.2f} - GENERATOR LOSS : {generator_loss} - CYCLELOSS : {cycle_loss}")

#     # Generating samples
#     generator_x2y.eval()
#     generator_y2x.eval()
#     discriminator_x.eval()
#     discriminator_y.eval()
#     with torch.no_grad():
#         print("[INFO] Creating samples")
#         food_samples = food_data[:3]
#         food2wine = generator_x2y(food_samples)
#         food_recovered = generator_y2x(generator_x2y(food_samples))

#         wine_samples = wine_data[:3]
#         wine2food = generator_y2x(wine_samples)

#         wine_recovered = generator_x2y(generator_y2x(wine_samples))

#     print(f"[DONE] Time elapsed: {time.time() - start_time:.2f} s")
#     return food_samples, food2wine, food_recovered, wine_samples, wine2food, wine_recovered

In [14]:
def train(config=None):
    with wandb.init(config=config):
        config = wandb.config

        food_data = df[food_columns]
        wine_data = df[wine_columns]

        n_epochs = config.epochs
        food_data = torch.from_numpy(food_data.to_numpy()).float().cuda()
        wine_data = torch.from_numpy(wine_data.to_numpy()).float().cuda()
        
        dataset_params = {'batch_size': 128, 'shuffle': False}
        food_loader = torch.utils.data.DataLoader(food_data, **dataset_params)
        wine_loader = torch.utils.data.DataLoader(wine_data, **dataset_params)
        
        generator_x2y = CycleGenerator(input_channels=len(food_columns), output_channels=len(wine_columns),hidden=config.generator).cuda()
        generator_y2x = CycleGenerator(input_channels=len(wine_columns), output_channels=len(food_columns),hidden=config.generator).cuda()
        discriminator_x = CycleDiscriminator(input_channels=len(food_columns),hidden=config.discriminator).cuda()
        discriminator_y = CycleDiscriminator(input_channels=len(wine_columns),hidden=config.discriminator).cuda()

        # generator_optimizer = torch.optim.Adam(list(generator_x2y.parameters()) + list(generator_y2x.parameters()), lr=2e-4,
        #                                        betas=(0.5, 0.999))
        generator_optimizer = optim.SGD(list(generator_x2y.parameters()) + list(generator_y2x.parameters()), lr=config.learning_rate_gen, momentum=0.9)                                    
        #discriminator_x_optimizer = torch.optim.Adam(discriminator_x.parameters(), lr=2e-4, betas=(0.5, 0.999))
        
        discriminator_x_optimizer = optim.SGD(discriminator_x.parameters(), lr=config.learning_rate_disc, momentum=0.9)   
        discriminator_y_optimizer = optim.SGD(discriminator_y.parameters(), lr=config.learning_rate_disc, momentum=0.9)   
        #discriminator_y_optimizer = torch.optim.Adam(discriminator_y.parameters(), lr=2e-4, betas=(0.5, 0.999))

        # Schedulers
        num_batches = len(food_loader)
        max_steps = max(1, n_epochs * num_batches)

        def scheduler(epoch):
            return max(0, (max_steps - epoch * num_batches) / max_steps)

        generator_scheduler = optim.lr_scheduler.LambdaLR(generator_optimizer, scheduler)
        discriminator_x_scheduler = optim.lr_scheduler.LambdaLR(discriminator_x_optimizer, scheduler)
        discriminator_y_scheduler = optim.lr_scheduler.LambdaLR(discriminator_y_optimizer, scheduler)
        mse_loss = nn.MSELoss()
        l1_loss = nn.L1Loss()
        zeros, ones = torch.tensor(0.).cuda(), torch.tensor(1.).cuda()
        cycle_lambda = config.cycle_lambda
        for epoch in range(n_epochs):
            for x_batch, y_batch in zip(food_loader, wine_loader):

                # Training generators x2y and y2x
                x_fake = generator_y2x(y_batch)
                d_x_fake = discriminator_x(x_fake)
                generator_x_loss = mse_loss(d_x_fake, ones)
                
                y_fake = generator_x2y(x_batch)
                d_y_fake = discriminator_y(y_fake)
                generator_y_loss = mse_loss(d_y_fake, ones)
                
                x_recovered = generator_y2x(y_fake)
                y_recovered = generator_x2y(x_fake)
                cycle_loss = l1_loss(x_recovered, x_batch) + l1_loss(y_recovered, y_batch)
                
                generator_loss = generator_x_loss + generator_y_loss + cycle_lambda * cycle_loss
                generator_optimizer.zero_grad()
                generator_loss.backward()
                generator_optimizer.step()                
                
                # Training discriminator for x
                x_fake = generator_y2x(y_batch)
                d_x_fake = discriminator_x(x_fake)
                d_x_true = discriminator_x(x_batch)
                discriminator_x_loss = mse_loss(d_x_true, ones ) + mse_loss(d_x_fake, zeros)
                
                discriminator_x_optimizer.zero_grad()
                discriminator_x_loss.backward()
                discriminator_x_optimizer.step()

                # Training discriminator for y
                y_fake = generator_x2y(x_batch)
                d_y_fake = discriminator_y(y_fake)
                d_y_true = discriminator_y(y_batch)
                discriminator_y_loss = mse_loss(d_y_true, ones) + mse_loss(d_y_fake, zeros)

                discriminator_y_optimizer.zero_grad()
                discriminator_y_loss.backward()
                discriminator_y_optimizer.step()
            generator_scheduler.step()
            discriminator_x_scheduler.step()
            discriminator_y_scheduler.step()
            print("epoch " ,epoch, "/",n_epochs)
            wandb.log({"cycle_loss": cycle_loss, "epoch": epoch, "generator_loss":generator_loss}) 

In [15]:
# wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: 0yhhbu6a with config:
wandb: 	criterion: mse
wandb: 	cycle_lambda: 10
wandb: 	discriminator: 30
wandb: 	epochs: 50
wandb: 	generator: 40
wandb: 	learning_rate_disc: 0.07232336977604145
wandb: 	learning_rate_gen: 0.097056828849458
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ramang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([67, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
epoch  0 / 50
epoch  1 / 50
epoch  2 / 50
epoch  3 / 50
epoch  4 / 50
epoch  5 / 50
epoch  6 / 50
epoch  7 / 50
epoch  8 / 50
epoch  9 / 50
epoch  10 / 50
epoch  11 / 50
epoch  12 / 50
epoch  13 / 50
epoch  14 / 50
epoch  15 / 50
epoch  16 / 50
epoch  17 / 50
epoch  18 / 50
epoch  19 / 50
epoch  20 / 50
ep

cycle_loss,0.46421
epoch,49
generator_loss,6.61491
_runtime,151
_timestamp,1620684703
_step,49


cycle_loss,█▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
generator_loss,▆▁▃▅▆▆▇▇▇▇▇▇▇▇███████████▇█▇█▇▇▇████▇██▇
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


wandb: Agent Starting Run: zhmqzvff with config:
wandb: 	criterion: mse
wandb: 	cycle_lambda: 40
wandb: 	discriminator: 40
wandb: 	epochs: 50
wandb: 	generator: 50
wandb: 	learning_rate_disc: 0.06198580693975796
wandb: 	learning_rate_gen: 0.027131425783157873
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

cycle_loss,▅▆█▆▅▃▄▃▂▂▃▃▂▃▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
generator_loss,▁▃▄▅▆▆▆▆▇▇▇▇▇▇▇▇████████████████████████
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


wandb: Agent Starting Run: 441az5vs with config:
wandb: 	criterion: mse
wandb: 	cycle_lambda: 10
wandb: 	discriminator: 50
wandb: 	epochs: 50
wandb: 	generator: 40
wandb: 	learning_rate_disc: 0.03075037849774667
wandb: 	learning_rate_gen: 0.05129945545131866
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


epoch  0 / 50
epoch  1 / 50
epoch  2 / 50
epoch  3 / 50
epoch  4 / 50
epoch  5 / 50
epoch  6 / 50
epoch  7 / 50
epoch  8 / 50
epoch  9 / 50
epoch  10 / 50
epoch  11 / 50
epoch  12 / 50
epoch  13 / 50
epoch  14 / 50
epoch  15 / 50
epoch  16 / 50
epoch  17 / 50
epoch  18 / 50
epoch  19 / 50
epoch  20 / 50
epoch  21 / 50
epoch  22 / 50
epoch  23 / 50
epoch  24 / 50
epoch  25 / 50
epoch  26 / 50
epoch  27 / 50
epoch  28 / 50
epoch  29 / 50
epoch  30 / 50
epoch  31 / 50
epoch  32 / 50
epoch  33 / 50
epoch  34 / 50
epoch  35 / 50
epoch  36 / 50
epoch  37 / 50
epoch  38 / 50
epoch  39 / 50
epoch  40 / 50
epoch  41 / 50
epoch  42 / 50
epoch  43 / 50
epoch  44 / 50
epoch  45 / 50
epoch  46 / 50
epoch  47 / 50
epoch  48 / 50
epoch  49 / 50


cycle_loss,0.46418
epoch,49
generator_loss,6.61229
_runtime,136
_timestamp,1620689686
_step,49


cycle_loss,▂▃▂▂▂▁▂█▁▁▁▂▁▁▂▁▁▁▁▂▂▁▂▂▃▃▂▃▂▃▂▃▃▂▃▂▂▁▂▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
generator_loss,▇████▁▆█████████████████████████████████
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


In [16]:
# wandb.agent(sweep_id, train, count=20)

wandb: Agent Starting Run: n8x4qngi with config:
wandb: 	criterion: mse
wandb: 	cycle_lambda: 50
wandb: 	discriminator: 50
wandb: 	epochs: 200
wandb: 	generator: 50
wandb: 	learning_rate_disc: 0.061333468947118025
wandb: 	learning_rate_gen: 0.09410891255064711
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


epoch  0 / 200
epoch  1 / 200
epoch  2 / 200
epoch  3 / 200
epoch  4 / 200
epoch  5 / 200
epoch  6 / 200
epoch  7 / 200
epoch  8 / 200
epoch  9 / 200
epoch  10 / 200
epoch  11 / 200
epoch  12 / 200
epoch  13 / 200
epoch  14 / 200
epoch  15 / 200
epoch  16 / 200
epoch  17 / 200
epoch  18 / 200
epoch  19 / 200
epoch  20 / 200
epoch  21 / 200
epoch  22 / 200
epoch  23 / 200
epoch  24 / 200
epoch  25 / 200
epoch  26 / 200
epoch  27 / 200
epoch  28 / 200
epoch  29 / 200
epoch  30 / 200
epoch  31 / 200
epoch  32 / 200
epoch  33 / 200
epoch  34 / 200
epoch  35 / 200
epoch  36 / 200
epoch  37 / 200
epoch  38 / 200
epoch  39 / 200
epoch  40 / 200
epoch  41 / 200
epoch  42 / 200
epoch  43 / 200
epoch  44 / 200
epoch  45 / 200
epoch  46 / 200
epoch  47 / 200
epoch  48 / 200
epoch  49 / 200
epoch  50 / 200
epoch  51 / 200
epoch  52 / 200
epoch  53 / 200
epoch  54 / 200
epoch  55 / 200
epoch  56 / 200
epoch  57 / 200
epoch  58 / 200
epoch  59 / 200
epoch  60 / 200
epoch  61 / 200
epoch  62 / 200
ep

cycle_loss,0.51113
epoch,199
generator_loss,27.55266
_runtime,531
_timestamp,1620690222
_step,199


cycle_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
generator_loss,▁▅▆▇▇▇▇▇▇███████████████████████████████
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: Agent Starting Run: a1froafh with config:
wandb: 	criterion: mse
wandb: 	cycle_lambda: 50
wandb: 	discriminator: 20
wandb: 	epochs: 100
wandb: 	generator: 20
wandb: 	learning_rate_disc: 0.07959994587741918
wandb: 	learning_rate_gen: 0.09692116081565506
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

cycle_loss,▄█▆▅▂▁▄▄▂▃▁▂▁▁▁▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
generator_loss,▁▆▇▇▇▇██████████████████████████████████
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: Agent Starting Run: 19ltcwse with config:
wandb: 	criterion: mse
wandb: 	cycle_lambda: 30
wandb: 	discriminator: 30
wandb: 	epochs: 500
wandb: 	generator: 20
wandb: 	learning_rate_disc: 0.007859410187786742
wandb: 	learning_rate_gen: 0.001501849112257947
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


epoch  0 / 500
epoch  1 / 500
epoch  2 / 500
epoch  3 / 500
epoch  4 / 500
epoch  5 / 500
epoch  6 / 500
epoch  7 / 500
epoch  8 / 500
epoch  9 / 500
epoch  10 / 500
epoch  11 / 500
epoch  12 / 500
epoch  13 / 500
epoch  14 / 500
epoch  15 / 500
epoch  16 / 500
epoch  17 / 500
epoch  18 / 500
epoch  19 / 500
epoch  20 / 500
epoch  21 / 500
epoch  22 / 500
epoch  23 / 500
epoch  24 / 500
epoch  25 / 500
epoch  26 / 500
epoch  27 / 500
epoch  28 / 500
epoch  29 / 500
epoch  30 / 500
epoch  31 / 500
epoch  32 / 500
epoch  33 / 500
epoch  34 / 500
epoch  35 / 500
epoch  36 / 500
epoch  37 / 500
epoch  38 / 500
epoch  39 / 500
epoch  40 / 500
epoch  41 / 500
epoch  42 / 500
epoch  43 / 500
epoch  44 / 500
epoch  45 / 500
epoch  46 / 500
epoch  47 / 500
epoch  48 / 500
epoch  49 / 500
epoch  50 / 500
epoch  51 / 500
epoch  52 / 500
epoch  53 / 500
epoch  54 / 500
epoch  55 / 500
epoch  56 / 500
epoch  57 / 500
epoch  58 / 500
epoch  59 / 500
epoch  60 / 500
epoch  61 / 500
epoch  62 / 500
ep

cycle_loss,0.23279
epoch,499
generator_loss,8.69954
_runtime,1327
_timestamp,1620703730
_step,499


cycle_loss,████▇▇▇▇▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
generator_loss,████▇▇▇▇▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [82]:
def train_cycle_gan():

    food_data = df[food_columns]
    wine_data = df[wine_columns]

    n_epochs = 500
    food_data = torch.from_numpy(food_data.to_numpy()).float().cuda()
    wine_data = torch.from_numpy(wine_data.to_numpy()).float().cuda()
    
    dataset_params = {'batch_size': 128, 'shuffle': False}
    food_loader = torch.utils.data.DataLoader(food_data, **dataset_params)
    wine_loader = torch.utils.data.DataLoader(wine_data, **dataset_params)
    
    generator_x2y = CycleGenerator(input_channels=len(food_columns), output_channels=len(wine_columns),hidden=20).cuda()
    generator_y2x = CycleGenerator(input_channels=len(wine_columns), output_channels=len(food_columns),hidden=20).cuda()
    discriminator_x = CycleDiscriminator(input_channels=len(food_columns),hidden=30).cuda()
    discriminator_y = CycleDiscriminator(input_channels=len(wine_columns),hidden=30).cuda()

    # generator_optimizer = torch.optim.Adam(list(generator_x2y.parameters()) + list(generator_y2x.parameters()), lr=2e-4,
    #                                        betas=(0.5, 0.999))
    generator_optimizer = optim.SGD(list(generator_x2y.parameters()) + list(generator_y2x.parameters()), lr=0.001501849112257947, momentum=0.9)                                    
    #discriminator_x_optimizer = torch.optim.Adam(discriminator_x.parameters(), lr=2e-4, betas=(0.5, 0.999))
    
    discriminator_x_optimizer = optim.SGD(discriminator_x.parameters(), lr=0.007859410187786742, momentum=0.9)   
    discriminator_y_optimizer = optim.SGD(discriminator_y.parameters(), lr=0.007859410187786742, momentum=0.9)   
    #discriminator_y_optimizer = torch.optim.Adam(discriminator_y.parameters(), lr=2e-4, betas=(0.5, 0.999))

    # Schedulers
    num_batches = len(food_loader)
    max_steps = max(1, n_epochs * num_batches)

    def scheduler(epoch):
        return max(0, (max_steps - epoch * num_batches) / max_steps)

    generator_scheduler = optim.lr_scheduler.LambdaLR(generator_optimizer, scheduler)
    discriminator_x_scheduler = optim.lr_scheduler.LambdaLR(discriminator_x_optimizer, scheduler)
    discriminator_y_scheduler = optim.lr_scheduler.LambdaLR(discriminator_y_optimizer, scheduler)
    mse_loss = nn.MSELoss()
    l1_loss = nn.L1Loss()
    zeros, ones = torch.tensor(0.).cuda(), torch.tensor(1.).cuda()
    cycle_lambda = 30 
   


    for epoch in range(n_epochs):
        for x_batch, y_batch in zip(food_loader, wine_loader):
            # Training generators x2y and y2x
            x_fake = generator_y2x(y_batch)
            d_x_fake = discriminator_x(x_fake)
            generator_x_loss = mse_loss(d_x_fake, ones)
            
            y_fake = generator_x2y(x_batch)
            d_y_fake = discriminator_y(y_fake)
            generator_y_loss = mse_loss(d_y_fake, ones)
            
            x_recovered = generator_y2x(y_fake)
            y_recovered = generator_x2y(x_fake)
            cycle_loss = l1_loss(x_recovered, x_batch) + l1_loss(y_recovered, y_batch)
            
            generator_loss = generator_x_loss + generator_y_loss + cycle_lambda * cycle_loss
            generator_optimizer.zero_grad()
            generator_loss.backward()
            generator_optimizer.step()                
            
            # Training discriminator for x
            x_fake = generator_y2x(y_batch)
            d_x_fake = discriminator_x(x_fake)
            d_x_true = discriminator_x(x_batch)
            discriminator_x_loss = mse_loss(d_x_true, ones ) + mse_loss(d_x_fake, zeros)
            
            discriminator_x_optimizer.zero_grad()
            discriminator_x_loss.backward()
            discriminator_x_optimizer.step()

            # Training discriminator for y
            y_fake = generator_x2y(x_batch)
            d_y_fake = discriminator_y(y_fake)
            d_y_true = discriminator_y(y_batch)
            discriminator_y_loss = mse_loss(d_y_true, ones) + mse_loss(d_y_fake, zeros)

            discriminator_y_optimizer.zero_grad()
            discriminator_y_loss.backward()
            discriminator_y_optimizer.step()
        generator_scheduler.step()
        discriminator_x_scheduler.step()
        discriminator_y_scheduler.step()

        print(f"[{100*(epoch+1)/n_epochs:.2f}%] Epoch {epoch + 1} - GENERATOR LOSS : {generator_loss} - CYCLELOSS : {cycle_loss}")

    # Generating samples
    generator_x2y.eval()
    generator_y2x.eval()
    discriminator_x.eval()
    discriminator_y.eval()
    with torch.no_grad():
        print("[INFO] Creating samples")
        food_samples = food_data[:3]
        food2wine = generator_x2y(food_samples)
        food_recovered = generator_y2x(generator_x2y(food_samples))

        wine_samples = wine_data[:3]
        wine2food = generator_y2x(wine_samples)

        wine_recovered = generator_x2y(generator_y2x(wine_samples))

    return food_samples, food2wine, food_recovered, wine_samples, wine2food, wine_recovered

In [83]:
food_samples, food2wine, food_recovered, wine_samples, wine2food, wine_recovered = train_cycle_gan()

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:446: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([67, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
[0.20%] Epoch 1 - GENERATOR LOSS : 15.020569801330566 - CYCLELOSS : 0.46389302611351013
[0.40%] Epoch 2 - GENERATOR LOSS : 13.461222648620605 - CYCLELOSS : 0.39623141288757324
[0.60%] Epoch 3 - GENERATOR LOSS : 15.10896110534668 - CYCLELOSS : 0.45051953196525574
[0.80%] Epoch 4 - GENERATOR LOSS : 13.324655

In [84]:
for i in range(3):
    print(food2wine[i])

tensor([0.1397, 0.0000, 0.0000, 0.5968, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.2745, 2.2022, 0.0000, 0.0000, 1.2661, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.5785, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.2177, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 1.2140], device='cuda:0')
tensor([0.1264, 0.0000, 0.0000, 2.9646, 0.0000, 0.0000, 0.0000, 0.0000, 0.5307,
        0.9688, 0.9419, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.6705, 0.0000, 0.0000,
        0.3347, 0.0000], device='cuda:0')
tensor([0.1123, 0.0000, 0.0000, 0.9964, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        1.5425, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.00

In [85]:
for i in range(3):
    print(wine_samples[i])

tensor([0.1650, 0.0033, 0.0572, 0.0000, 3.3000, 0.0000, 0.0000, 2.2000, 0.0000,
        0.0000, 1.1000, 0.0000, 1.1000, 2.2000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.2000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.1000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor([0.1500, 0.0030, 0.0520, 0.0000, 3.0000, 0.0000, 0.0000, 2.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 2.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor([1.5900e-01, 3.1800e-03, 5.5120e-02, 0.0000e+00, 3.1800e+00, 0.0000e+00,
        0.0000e+00, 2.1200e+00, 0.0000e+00, 0.0000e+00, 1.0600e+00, 0.0000e+00,
        1.0600e+00, 2.1200e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.00

In [73]:
def train_cycle_gan2():

    food_data = df[food_columns][:3]
    wine_data = df[wine_columns][:3]

    n_epochs = 100
    food_data = torch.from_numpy(food_data.to_numpy()).float().cuda()
    wine_data = torch.from_numpy(wine_data.to_numpy()).float().cuda()
    
    dataset_params = {'batch_size': 1, 'shuffle': False}
    food_loader = torch.utils.data.DataLoader(food_data, **dataset_params)
    wine_loader = torch.utils.data.DataLoader(wine_data, **dataset_params)
    
    generator_x2y = CycleGenerator(input_channels=len(food_columns), output_channels=len(wine_columns),hidden=20).cuda()
    generator_y2x = CycleGenerator(input_channels=len(wine_columns), output_channels=len(food_columns),hidden=20).cuda()
    discriminator_x = CycleDiscriminator(input_channels=len(food_columns),hidden=30).cuda()
    discriminator_y = CycleDiscriminator(input_channels=len(wine_columns),hidden=30).cuda()

    # generator_optimizer = torch.optim.Adam(list(generator_x2y.parameters()) + list(generator_y2x.parameters()), lr=2e-4,
    #                                        betas=(0.5, 0.999))
    generator_optimizer = optim.SGD(list(generator_x2y.parameters()) + list(generator_y2x.parameters()), lr=0.001501849112257947, momentum=0.9)                                    
    #discriminator_x_optimizer = torch.optim.Adam(discriminator_x.parameters(), lr=2e-4, betas=(0.5, 0.999))
    
    discriminator_x_optimizer = optim.SGD(discriminator_x.parameters(), lr=0.007859410187786742, momentum=0.9)   
    discriminator_y_optimizer = optim.SGD(discriminator_y.parameters(), lr=0.007859410187786742, momentum=0.9)   
    #discriminator_y_optimizer = torch.optim.Adam(discriminator_y.parameters(), lr=2e-4, betas=(0.5, 0.999))

    # Schedulers
    num_batches = len(food_loader)
    max_steps = max(1, n_epochs * num_batches)

    def scheduler(epoch):
        return max(0, (max_steps - epoch * num_batches) / max_steps)

    generator_scheduler = optim.lr_scheduler.LambdaLR(generator_optimizer, scheduler)
    discriminator_x_scheduler = optim.lr_scheduler.LambdaLR(discriminator_x_optimizer, scheduler)
    discriminator_y_scheduler = optim.lr_scheduler.LambdaLR(discriminator_y_optimizer, scheduler)
    mse_loss = nn.MSELoss()
    l1_loss = nn.L1Loss()
    zeros, ones = torch.tensor(0.).cuda(), torch.tensor(1.).cuda()
    cycle_lambda = 30
   


    for epoch in range(n_epochs):
        for x_batch, y_batch in zip(food_loader, wine_loader):
            # Training generators x2y and y2x
            x_fake = generator_y2x(y_batch)
            d_x_fake = discriminator_x(x_fake)
            generator_x_loss = mse_loss(d_x_fake, ones)
            
            y_fake = generator_x2y(x_batch)
            d_y_fake = discriminator_y(y_fake)
            generator_y_loss = mse_loss(d_y_fake, ones)
            
            x_recovered = generator_y2x(y_fake)
            y_recovered = generator_x2y(x_fake)
            cycle_loss = l1_loss(x_recovered, x_batch) + l1_loss(y_recovered, y_batch)
            
            generator_loss = generator_x_loss + generator_y_loss + cycle_lambda * cycle_loss
            generator_optimizer.zero_grad()
            generator_loss.backward()
            generator_optimizer.step()                
            
            # Training discriminator for x
            x_fake = generator_y2x(y_batch)
            d_x_fake = discriminator_x(x_fake)
            d_x_true = discriminator_x(x_batch)
            discriminator_x_loss = mse_loss(d_x_true, ones ) + mse_loss(d_x_fake, zeros)
            
            discriminator_x_optimizer.zero_grad()
            discriminator_x_loss.backward()
            discriminator_x_optimizer.step()

            # Training discriminator for y
            y_fake = generator_x2y(x_batch)
            d_y_fake = discriminator_y(y_fake)
            d_y_true = discriminator_y(y_batch)
            discriminator_y_loss = mse_loss(d_y_true, ones) + mse_loss(d_y_fake, zeros)

            discriminator_y_optimizer.zero_grad()
            discriminator_y_loss.backward()
            discriminator_y_optimizer.step()
        generator_scheduler.step()
        discriminator_x_scheduler.step()
        discriminator_y_scheduler.step()

        print(f"[{100*(epoch+1)/n_epochs:.2f}%] Epoch {epoch + 1} - GENERATOR LOSS : {generator_loss} - CYCLELOSS : {cycle_loss}")

    # Generating samples
    generator_x2y.eval()
    generator_y2x.eval()
    discriminator_x.eval()
    discriminator_y.eval()
    with torch.no_grad():
        print("[INFO] Creating samples")
        food_samples = food_data[:3]
        food2wine = generator_x2y(food_samples)
        food_recovered = generator_y2x(generator_x2y(food_samples))

        wine_samples = wine_data[:3]
        wine2food = generator_y2x(wine_samples)

        wine_recovered = generator_x2y(generator_y2x(wine_samples))

    return food_samples, food2wine, food_recovered, wine_samples, wine2food, wine_recovered

In [80]:
food_samples, food2wine, food_recovered, wine_samples, wine2food, wine_recovered = train_cycle_gan2()

[1.00%] Epoch 1 - GENERATOR LOSS : 20.844619750976562 - CYCLELOSS : 0.6798557043075562
[2.00%] Epoch 2 - GENERATOR LOSS : 20.60897445678711 - CYCLELOSS : 0.6719324588775635
[3.00%] Epoch 3 - GENERATOR LOSS : 20.234848022460938 - CYCLELOSS : 0.659365713596344
[4.00%] Epoch 4 - GENERATOR LOSS : 19.8828067779541 - CYCLELOSS : 0.6475248336791992
[5.00%] Epoch 5 - GENERATOR LOSS : 19.63088035583496 - CYCLELOSS : 0.6390105485916138
[6.00%] Epoch 6 - GENERATOR LOSS : 19.46849822998047 - CYCLELOSS : 0.6334773302078247
[7.00%] Epoch 7 - GENERATOR LOSS : 19.36381721496582 - CYCLELOSS : 0.6298696994781494
[8.00%] Epoch 8 - GENERATOR LOSS : 19.39195442199707 - CYCLELOSS : 0.630681037902832
[9.00%] Epoch 9 - GENERATOR LOSS : 19.538188934326172 - CYCLELOSS : 0.6354383230209351
[10.00%] Epoch 10 - GENERATOR LOSS : 19.69139862060547 - CYCLELOSS : 0.6404397487640381
[11.00%] Epoch 11 - GENERATOR LOSS : 19.8663272857666 - CYCLELOSS : 0.6461734771728516
[12.00%] Epoch 12 - GENERATOR LOSS : 20.04519271850

In [81]:
for i in range(3):
    print(food2wine[i])


tensor([0.0000e+00, 0.0000e+00, 3.3542e-01, 3.6916e-03, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.5567e+00, 0.0000e+00,
        0.0000e+00, 3.8889e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        7.6166e-02, 0.0000e+00, 0.0000e+00, 1.4421e-01, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00], device='cuda:0')
tensor([0.0000, 0.0000, 0.3344, 0.0000, 0.0000, 0.0999, 0.0000, 0.0000, 0.0000,
        0.0000, 1.4185, 0.0000, 0.0000, 3.7346, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0459, 0.0000, 0.0000, 0.2162, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0023, 0.0000, 0.0000, 0.0000, 0.0098,
        0.0000, 0.7176, 0.00

In [72]:
for i in range(3):
    print(wine_samples[i])

tensor([0.1650, 0.0033, 0.0572, 0.0000, 3.3000, 0.0000, 0.0000, 2.2000, 0.0000,
        0.0000, 1.1000, 0.0000, 1.1000, 2.2000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.2000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.1000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor([0.1500, 0.0030, 0.0520, 0.0000, 3.0000, 0.0000, 0.0000, 2.0000, 0.0000,
        0.0000, 1.0000, 0.0000, 1.0000, 2.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 2.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000], device='cuda:0')
tensor([1.5900e-01, 3.1800e-03, 5.5120e-02, 0.0000e+00, 3.1800e+00, 0.0000e+00,
        0.0000e+00, 2.1200e+00, 0.0000e+00, 0.0000e+00, 1.0600e+00, 0.0000e+00,
        1.0600e+00, 2.1200e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.00